In [1]:
#Example Notebook
from sklearn.datasets import load_iris
from datetime import datetime
from refit.api.refit import Refit
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from refit.enums.ModelFormat import ModelFormat
from skl2onnx.common.data_types import FloatTensorType, BooleanTensorType


In [2]:
iris = load_iris()

In [3]:
project_guid = "b6ee5bab-08dd-49b0-98b6-45cd0a28b12f"
refit = Refit(project_guid)

start = datetime(2020, 6, 27)
end = datetime(2020, 6, 29)
sensors = list(
    map(lambda x: str(x),
    range(5160, 5170))) #['5163']

sensors

/Users/matthewalvarez/Documents/projects/iot-prototype/training/refit/util/Schema.py:23: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict = yaml.load(schema_yaml)


['5160',
 '5161',
 '5162',
 '5163',
 '5164',
 '5165',
 '5166',
 '5167',
 '5168',
 '5169']

In [4]:
# Getting Sensor Data with the training window flag incorporated
df = refit.sensor_data_with_flag(start, end, sensors=sensors)
df

,sensor_id,partition_key,timestamp,temperature,pressure,wind,operable
0,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,True
1,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,True
2,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,True
3,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,True
4,5163,2020-06-28 00,2020-06-28 00:48:42,68.0,1028.0,64.0,True
...,...,...,...,...,...,...,...
155,5169,2020-06-28 04,2020-06-28 04:34:16,78.0,1031.0,19.0,False
156,5169,2020-06-28 04,2020-06-28 04:34:21,69.0,1174.0,70.0,False
157,5169,2020-06-28 04,2020-06-28 04:34:26,30.0,914.0,47.0,False
158,5169,2020-06-28 04,2020-06-28 04:34:31,41.0,966.0,12.0,False


In [5]:
skip_columns = ['project_guid', 'sensor_id', 'partition_key', 'timestamp', 'operable']
feature_columns = list(filter( lambda x: x not in skip_columns, df.columns.array))
feature_columns

['temperature', 'pressure', 'wind']

In [6]:
df['temperature'] = df['temperature'].astype(float)
df['pressure'] = df['pressure'].astype(float)
df['wind'] = df['wind'].astype(float)

df['operable'] = (df.temperature < 75.0)
df['operable'] = df['operable'].astype(int)
df

,sensor_id,partition_key,timestamp,temperature,pressure,wind,operable
0,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,1
1,5163,2020-06-28 00,2020-06-28 00:48:32,60.0,1107.0,38.0,1
2,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,1
3,5163,2020-06-28 00,2020-06-28 00:48:37,43.0,1152.0,14.0,1
4,5163,2020-06-28 00,2020-06-28 00:48:42,68.0,1028.0,64.0,1
...,...,...,...,...,...,...,...
155,5169,2020-06-28 04,2020-06-28 04:34:16,78.0,1031.0,19.0,0
156,5169,2020-06-28 04,2020-06-28 04:34:21,69.0,1174.0,70.0,1
157,5169,2020-06-28 04,2020-06-28 04:34:26,30.0,914.0,47.0,1
158,5169,2020-06-28 04,2020-06-28 04:34:31,41.0,966.0,12.0,1


In [7]:
x = df[feature_columns]
y = df['operable']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3) # 70% training and 30% test

In [9]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

RandomForestClassifier()

In [10]:
y_pred=clf.predict(X_test)
y_pred

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1])

In [11]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [25]:
initial_types = [('input', FloatTensorType([None,len(feature_columns)]))]

In [26]:
bytes = refit.save(clf, ModelFormat.SK_LEARN, initial_types=initial_types)

ConnectionError: HTTPConnectionPool(host='localhost', port=3030): Max retries exceeded with url: /models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x131ef5940>: Failed to establish a new connection: [Errno 61] Connection refused'))